In [1]:
import numpy as np
import pandas as pd
import matplotlib
# %%local
# %matplotlib inline
import matplotlib.pyplot as plt
# %matplot plt
# plt.show()
# from IPython import get_ipython
# get_ipython().run_line_magic('matplotlib', 'inline')
# get_ipython().magic(u'matplotlib inline')
import seaborn as sns

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark").getOrCreate()

# spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1682827100408_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.evaluation import ClusteringEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def printdf(df, l=5):
    return df.limit(l).toPandas()

def nullcount(df):
    return {col: df.filter(df[col].isNull()).count() for col in df.columns}

def shape(df):
    print((df.count(), len(df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Load data
**dataset from RFM with PySpark**

In [23]:
rfm_numbers = spark.read.csv(f"s3://athenapyspark-rfm-analysis-result/boost-analysis/evo_rfm_numbers.csv", 
                             inferSchema=True, 
                             header=True)
printdf(rfm_numbers)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   user_id  Recency  Frequency  Monetary
0        5       59         19  17923.42
1       31       81         10   3181.85
2       75      295          2    765.00
3       89      356          1    500.00
4      144       72         17  10750.06

<!-- # Prepare data -->

# Inspect

In [24]:
df_rfm_numbers = rfm_numbers.toPandas()
df_rfm_numbers.describe()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          user_id     Recency  Frequency      Monetary
count   36.000000   36.000000  36.000000     36.000000
mean   329.500000  128.527778   8.361111   5370.250303
std    175.338448   93.640494   9.553317   8102.575031
min      5.000000   59.000000   1.000000      0.020000
25%    191.250000   68.500000   2.000000    400.000000
50%    320.000000   75.000000   5.500000   1128.150000
75%    510.250000  172.500000  11.000000   8122.952500
max    597.000000  366.000000  41.000000  36631.610000

In [25]:
fig, ax = plt.subplots(1, 3, figsize=(16, 8))

# Recency distribution plot
sns.histplot(df_rfm_numbers['Recency'], kde=True, ax=ax[0])

# Frequency distribution plot
sns.histplot(df_rfm_numbers.query('Frequency < 1000')['Frequency'], kde=True, ax=ax[1])

# Monetary distribution plot
sns.histplot(df_rfm_numbers.query('Monetary < 10000')['Monetary'], kde=True, ax=ax[2])

import boto3
s3client=boto3.client('s3')
fig.savefig('kmeans_distribution.png')
s3client.upload_file('kmeans_distribution.png', 'evo-kmeans-analysis', 'boost-analysis/analysis-results/kmeans_distribution.png')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# K-Means Clustering

In [26]:
# Since log transformation doesn't work with negative and zero values, lets get rid of negative and zeros

rfm_data = (
    rfm_numbers.withColumn(
        "Monetary", 
        F.when(F.col("Monetary") <= 0, 1)
         .otherwise(F.col("Monetary")))
)

# rfm_data = rfm_numbers
# printdf(rfm_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Elbow Analysis

In [27]:
features = rfm_data.columns[1:]

printdf(rfm_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   user_id  Recency  Frequency  Monetary
0        5       59         19  17923.42
1       31       81         10   3181.85
2       75      295          2    765.00
3       89      356          1    500.00
4      144       72         17  10750.06

In [28]:
assembler = VectorAssembler(
    inputCols=features, 
    outputCol="rfm_features")

assembled_data = assembler.transform(rfm_data)

assembled_data = assembled_data.select('user_id', 'rfm_features')

printdf(assembled_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   user_id                      rfm_features
0        5  [59.0, 19.0, 17923.420000000002]
1       31  [81.0, 10.0, 3181.8500000000004]
2       75               [295.0, 2.0, 765.0]
3       89               [356.0, 1.0, 500.0]
4      144            [72.0, 17.0, 10750.06]

In [29]:
scaler = StandardScaler(inputCol='rfm_features', outputCol='rfm_standardized')

data_scale = scaler.fit(assembled_data)

scaled_data = data_scale.transform(assembled_data)

printdf(scaled_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   user_id  ...                                   rfm_standardized
0        5  ...  [0.6300692965006142, 1.988837933331324, 2.2120...
1       31  ...  [0.8650103901110127, 1.0467568070164863, 0.392...
2       75  ...  [3.1503464825030707, 0.20935136140329727, 0.09...
3       89  ...  [3.8017740602409935, 0.10467568070164864, 0.06...
4      144  ...  [0.7688981245431223, 1.7794865719280268, 1.326...

[5 rows x 3 columns]

In [30]:
# working with `scaled_data`
# from pyspark.ml.evaluation import ClusteringEvaluator
costs = {}

for k in range(2, 10):
    k_means = KMeans(featuresCol='rfm_standardized', k=k)
    model = k_means.fit(scaled_data)
    costs[k] = model.summary.trainingCost

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
fig, ax = plt.subplots(1, 1, figsize =(16, 8))

ax.plot(costs.keys(), costs.values(), '-ob')
ax.set_xlabel('k')
ax.set_ylabel('cost')

# ax.plot(3, costs[3], 'bo')

import boto3
s3client=boto3.client('s3')
fig.savefig('kmeans_cluster_number.png')
s3client.upload_file('kmeans_cluster_number.png', 'evo-kmeans-analysis', 'boost-analysis/analysis-results/kmeans_cluster_number.png')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# K = 3

In [32]:
k_means = KMeans(featuresCol='rfm_standardized', k=3)
model = k_means.fit(scaled_data)
predictions = model.transform(scaled_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
result = predictions.select('user_id', 'prediction')
printdf(result)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   user_id  prediction
0        5           1
1       31           2
2       75           0
3       89           0
4      144           1

In [35]:
# Join other information with the prediction result-set

rfm_score = spark.read.csv(f's3://athenapyspark-rfm-analysis-result/analysis-results/evo_loyalty_rfm.csv', 
                           inferSchema=True, 
                           header=True)
rfm_score = rfm_score.select("user_id", "Recency", "Frequency", "Monetary", "RFM_Score", "RFM_ScoreGroup", "Loyalty")
# printdf(rfm_score)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
combined_result = result.join(rfm_score, on='user_id', how='inner')
printdf(combined_result)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   user_id  prediction  Recency  ...  RFM_Score  RFM_ScoreGroup   Loyalty
0      242           1       74  ...          3             111  Platinum
1      280           1      116  ...          3             111  Platinum
2      324           2      131  ...          3             111  Platinum
3      351           2       63  ...          3             111  Platinum
4      378           1       67  ...          3             111  Platinum

[5 rows x 8 columns]

In [37]:
combined_result_df = combined_result.toPandas()
import boto3
s3 = boto3.resource('s3')
bucket = s3.Bucket('evo-kmeans-analysis')
import tempfile
with tempfile.TemporaryFile(mode='w+t') as f:
    # write the DataFrame to the temporary file
    combined_result_df.to_csv(f, index=False)
    # reset the file pointer to the beginning of the file
    f.seek(0)
    # upload the file to S3
    bucket.put_object(Key='boost-analysis/analysis-results/evo-combined_result_k4.csv', Body=f.read().encode())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0
s3.Object(bucket_name='evo-kmeans-analysis', key='boost-analysis/analysis-results/evo-combined_result_k4.csv')

In [38]:
# inspect our loyalty segmentation

# combined_result_df = combined_result_df.sort_values("RFM_Score")

fig, ax = plt.subplots(1, 1, figsize=(6, 8))

sns.countplot(x="prediction", data=combined_result_df)
for p in ax.patches:
    ax.annotate('{}'.format(p.get_height()), (p.get_x() + 0.3, p.get_height() + 20))
    
import boto3
s3client=boto3.client('s3')
fig.savefig('kmeans_prediction.png')
s3client.upload_file('kmeans_prediction.png', 'evo-kmeans-analysis', 'boost-analysis/analysis-results/kmeans_prediction.png')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Observation

In [39]:
analysis_df = combined_result.toPandas()

fig, ax = plt.subplots(1, 3, figsize=(20, 12))
sns.boxplot(x='prediction', y='Recency', data=analysis_df, ax=ax[0])
sns.boxplot(x='prediction', y='Frequency', data=analysis_df, ax=ax[1])
sns.boxplot(x='prediction', y='Monetary', data=analysis_df, ax=ax[2])

import boto3
s3client=boto3.client('s3')
fig.savefig('kmeans_observation.png')
s3client.upload_file('kmeans_observation.png', 'evo-kmeans-analysis', 'boost-analysis/analysis-results/kmeans_observation.png')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Visualization

In [40]:
selected_result_df = combined_result_df#.query('Monetary < 100000').query('Frequency < 3000')
import boto3
s3client=boto3.client('s3')

# Monetary vs Recency (combined)
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
sns.scatterplot(x='Recency', y='Monetary', data=selected_result_df, hue='prediction', palette="deep")
fig.savefig('kmeans_Monetary_vs_Recency.png')
s3client.upload_file('kmeans_Monetary_vs_Recency.png', 'evo-kmeans-analysis', 'boost-analysis/analysis-results/kmeans_Monetary_vs_Recency.png')

# Recency vs Frequency (combined)
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
sns.scatterplot(x='Recency', y='Frequency', data=selected_result_df, hue='prediction', palette="deep")
fig.savefig('kmeans_Recency_vs_Frequency.png')
s3client.upload_file('kmeans_Recency_vs_Frequency.png', 'evo-kmeans-analysis', 'boost-analysis/analysis-results/kmeans_Recency_vs_Frequency.png')

# Monetary vs Frequency (combined)
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
sns.scatterplot(x='Monetary', y='Frequency', data=selected_result_df, hue='prediction', palette="deep")
fig.savefig('kmeans_Monetary_vs_Frequency.png')
s3client.upload_file('kmeans_Monetary_vs_Frequency.png', 'evo-kmeans-analysis', 'boost-analysis/analysis-results/kmeans_Monetary_vs_Frequency.png')



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
analysis = combined_result\
    .groupBy('prediction')\
    .agg(F.avg('Recency').alias('Avg Recency'),
         F.avg('Frequency').alias('Avg Frequency'),
         F.avg('Monetary').alias('Avg Monetary'))
printdf(analysis)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   prediction  Avg Recency  Avg Frequency  Avg Monetary
0           1   145.625000      13.250000  16141.461250
1           2   125.954545       6.318182   4371.146136
2           0   235.500000       2.500000   1904.366667